In [142]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
from envs.GraphEnv.impnode import ImpnodeEnv
from envs.sync_vector_env import GraphSyncVectorEnv
from envs.async_vector_env import GraphAsyncVectorEnv
from envs.async_vector_env2 import GraphAsyncVectorEnv2
import  numpy as np
import time

In [163]:
def make_env_fn(seed):
    env = ImpnodeEnv(ba_nodes = 200, ba_edges = 50, max_removed_nodes = 10, seed = seed)
    return env

num_envs = 5000
num_episodes = 1

In [164]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [165]:
def sync_func(num_envs, num_episodes):
    sync_envs = GraphSyncVectorEnv([make_env_fn(i) for i in range(num_envs)])
    for i in range(num_envs):
        sync_envs.action_space(i).seed(i)

    for i in range(num_episodes):
        terminated = False
        num_nodes = [sync_envs.num_nodes(i) for i in range(num_envs)]
        masks = [np.ones(nodes, dtype=np.int8) for nodes in num_nodes]
        while not terminated:
            actions = [sync_envs.action_space(i).sample(mask=masks[i]) for i in range(num_envs)]
            for j in range(num_envs):
                masks[j][actions[j]] = 0
            start = time.time()
            observation, reward, terminated, truncated, info = sync_envs.step(actions)
            end = time.time()
            terminated = sum(terminated)
        sync_envs.reset()
    print(end-start)

In [166]:
%lprun -f sync_func sync_func(num_envs, num_episodes)

965.3215310573578


In [167]:
def async_func(num_envs, num_episodes):
    async_envs = GraphAsyncVectorEnv([make_env_fn(i) for i in range(num_envs)])
    for i in range(num_envs):
        async_envs.action_space(i).seed(i)
    if __name__ == '__main__':
        for i in range(num_episodes):
            terminated = False
            num_nodes = [async_envs.num_nodes(i) for i in range(num_envs)]
            masks = [np.ones(nodes, dtype=np.int8) for nodes in num_nodes]
            while not terminated:
                actions = [async_envs.action_space(i).sample(mask=masks[i]) for i in range(num_envs)]
                for j in range(num_envs):
                    masks[j][actions[j]] = 0
                start = time.time()
                observation, reward, terminated, truncated, info = async_envs.step(actions)
                end = time.time()
                terminated = sum(terminated)
            async_envs.reset()
        print(end-start)

In [ ]:
%lprun -f async_func async_func(num_envs, num_episodes)

In [ ]:
async_envs = GraphAsyncVectorEnv2([make_env_fn(i) for i in range(num_envs)])
for i in range(num_envs):
    async_envs.action_space(i).seed(i)
if __name__ == '__main__':
    for i in range(num_episodes):
        terminated = False
        num_nodes = [async_envs.num_nodes(i) for i in range(num_envs)]
        masks = [np.ones(nodes, dtype=np.int8) for nodes in num_nodes]
        while not terminated:
            actions = [async_envs.action_space(i).sample(mask=masks[i]) for i in range(num_envs)]
            for j in range(num_envs):
                masks[j][actions[j]] = 0
            observation, reward, terminated, truncated, info = async_envs.step(actions)
            terminated = sum(terminated)
        async_envs.reset()